# Trouver et appliquer les règles à entropie nulle


## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [1]:
# -*- coding: utf8 -*-
import codecs,glob
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import networkx as nx
import pickle,yaml
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

### Préparation des matrices de traits

In [2]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [4]:
ipaIn = listerUnicode(u"SZNêôârEHO")
ipaOut= [u"ʃ",u"ʒ",u"ŋ",u"ɛ̃",u"ɔ̃",u"ɑ̃",u"ʁ",u"ɛ",u"ɥ",u"ɔ"]
toipa = dict(zip(ipaIn, ipaOut))

S Z N ê ô â r E H O


In [5]:
import unicodedata
def listerUnicode(chaine):
    result=[]
    comb=False
    prec=u""
    for char in chaine:
        print char,
        if unicodedata.combining(char):
            result.append(prec+char)
            prec=""
        else:
            result.append(prec)
            prec=char
    result.append(prec)
    return [r for r in result if r!=""]

def coderIPA(chaine,table=toipa):
    result=chaine
    for k in table:
        result=result.replace(k,table[k])
    return result

In [6]:
print coderIPA(u"SâZ piNpôN")

ʃɑ̃ʒ piŋpɔ̃ŋ


In [8]:
rep="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Longitudinales/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/StemSpaces/LongitudinalesRnd/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/StemSpaces/Longitudinales/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/lexique3/"
rep="/Users/gilles/Desktop/lexique3/"
fichiers=glob.glob(rep+"*-X*-Regles.pkl")
samples=[f.rsplit("/",1)[-1].split("Regles")[0] for f in fichiers]
# samples={int(s.split("-")[1]):s for s in samples}
samples={int(s.split("-")[2]):s for s in samples}
samples

{0: 'Longitudinal-Lexique3-00-T150000000-F73539-X-Morphomes-',
 1: 'Longitudinal-Lexique3-01-T150000000-F254455-X-Morphomes-'}

In [9]:
sample=samples[1]
fRulesPMS="Regles.pkl"
fRulesPMO="Morphomes-Regles.pkl"

# Classe pour la gestion des patrons, des classes et des transformations

In [10]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print "le numéro de forme n'est pas dans [1,2]",n

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=False):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
#                if debug: 
#                    print (forme, file=logfile)
#                    print ("pas de classe",idClasseForme, file=logfile)
#                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
#        else:
#            if debug:
#                print (forme, file=logfile) 
#                print ("pas de patron", file=logfile)
        return sortieForme
        

## Ouvrir les fichiers de règles

In [11]:
def openRules(numero,typeRules="PMS"):
    rules=None
    if typeRules=="PMS" and numero in samples:
        with open(rep+samples[numero]+fRulesPMS, 'rb') as input:
            rules = pickle.load(input)
    elif typeRules=="PMO" and numero in samples:
        with open(rep+samples[numero]+fRulesPMO, 'rb') as input:
            rules = pickle.load(input)
    return rules

In [12]:
numero=1
typeRules="PMO"
avecContexte=True

In [14]:
sample

'Longitudinal-Lexique3-01-T150000000-F254455-X-Morphomes-'

In [18]:
#rules=openRules(numero,typeRules=typeRules)
with open(rep+sample+"Regles.pkl", 'rb') as input:
    rules = pickle.load(input)

In [38]:
rules[("ii1S","ii1P")].patrons

{u'E-ij\xf4': u'^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZ][lr])E$',
 u'E-i\xf4': u'^(.*[uOo][fv])rE$',
 u'E-j\xf4': u'^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZmnJjlrE6\xea])E$',
 u'E-\xf4': u'^(.*[jrwHiyEe926auOo][Jj])E$',
 u'HE-j\xf4': u'^(.*)yHE$',
 u'isE-j\xf4': u'^(.*)sOrtisE$',
 u'jE-ij\xf4': u'^(.*[ptkbdgfsSvzZmnJlr])jE$',
 u'wE-j\xf4': u'^(.*)uwE$',
 u'wajE-Ej\xf4': u'^(.*[rE6a])swajE$'}

In [21]:
zeroEntropie=[]
for paire in rules:
    classe=rules[paire].classe
    c1,c2=paire
    if c1!=c2:
        zero=True
        print paire
        for transformation in classe:
            choix=classe[transformation]
            if len(choix)>1:
                print choix
                zero=False
        if zero:
            print "zéro",choix
            zeroEntropie.append(paire)
        print

(u'pI2S', u'ps2P')
{u'-je': 1, u'-sje': 13}
{u'wa-6vje': 9, u'-je': 14}
{u'-tje': 7, u'-je': 136}
{u'-je': 50, u'E.-6.je': 78}
{u'-dje': 27, u'-tje': 5}

(u'ai1S', u'pc2P')
{u'6.E-E.6rje': 146, u'E-6rje': 1}
{u'y-6vrije': 9, u'y-Orje': 2}
{u'jE-rje': 67, u'E-6rje': 169}
{u'i-Etrije': 15, u'-rje': 5}
{u'i-rije': 3, u'-rje': 3, u'i-Erje': 1}
{u'y-Etrije': 9, u'y-warje': 6}
{u'i-6rje': 7, u'-rje': 1}
{u'i-rije': 55, u'-rje': 74}

(u'is1P', u'ii2P')
zéro {u'is.\xf4-6n.e': 11}

(u'ai2S', u'fi1P')
{u'-r\xf4': 123, u'i-r\xf4': 58}
{u'-r\xf4': 5, u'y-ir\xf4': 4, u'y-Er\xf4': 3}
{u'a-6r\xf4': 16, u'6.a-E.6r\xf4': 151}
{u'a-6r\xf4': 174, u'ja-r\xf4': 97}
{u'-r\xf4': 4, u'i-6r\xf4': 4}
{u'-r\xf4': 2, u'i-war\xf4': 1, u'i-jEr\xf4': 1}
{u'y-ir\xf4': 1, u'ly-dr\xf4': 3}
{u'y-6vr\xf4': 7, u'y-Or\xf4': 2}
{u'-r\xf4': 11, u'i-Etr\xf4': 15}
{u'i-r\xf4': 2, u'i-Err\xf4': 2}
{u'-r\xf4': 12, u'i-war\xf4': 2, u'i-jEr\xf4': 2}
{u'y-war\xf4': 3, u'y-watr\xf4': 3}
{u'y-Etr\xf4': 1, u'y-ur\xf4': 1}
{u'-r\xf4': 

{u'j.-r.': 77, u'\xf4-6r\xf4': 167}
{u'z.-r.': 7, u'6.\xf4-E.6r\xf4': 3}
{u'\xf4-r\xf4': 9, u'6.\xf4-E.6r\xf4': 51, u'\xf4-6r\xf4': 6}
{u'6.\xf4-E.6r\xf4': 16, u'\xf4-6r\xf4': 1}
{u'av.-Or.': 1, u'\xf4-6r\xf4': 5}
{u'\xf4-ir\xf4': 4, u'\xf4-6r\xf4': 148}
{u's.-r.': 280, u'\xf4-6r\xf4': 29}
{u'z.-r.': 1, u'\xf4-6r\xf4': 7}
{u'6.\xf4-E.6r\xf4': 17, u'\xf4-6r\xf4': 2}
{u'z.-r.': 38, u'\xf4-6r\xf4': 200}
{u'6n\xf4-j\xeadr\xf4': 27, u'6.\xf4-E.6r\xf4': 1}
{u'6.\xf4-E.6r\xf4': 63, u'\xf4-6r\xf4': 2}
{u'av.-Or.': 2, u'\xf4-6r\xf4': 12}
{u'Ej\xf4-jEr\xf4': 3, u'Ej\xf4-war\xf4': 3, u'j.-r.': 4, u'\xf4-6r\xf4': 4}
{u'al\xf4-Odr\xf4': 5, u'\xf4-6r\xf4': 7}
{u'\xf4-r\xf4': 26, u'\xf4-6r\xf4': 1857}
{u'j.-r.': 2, u'waj.-jEr.': 2}
{u'\xf4-r\xf4': 66, u'\xf4-ir\xf4': 32, u'\xf4-6r\xf4': 758}
{u'\xf4-6r\xf4': 32, u's\xf4-tr\xf4': 14}
{u'v.-r.': 13, u'\xf4-r\xf4': 3, u'\xf4-6r\xf4': 14}
{u'al.-ir.': 2, u'\xf4-6r\xf4': 1}
{u'waj\xf4-Er\xf4': 12, u'j.-r.': 10}

(u'pc1P', u'ps2P')
{u'E.6rj\xf4-6.je': 78, 

{u'-je': 51, u'E-ave': 1}
{u'-e': 7, u'Ej-waje': 2}

(u'pc1S', u'pI2S')
{u'rE-j': 13, u'rE-': 14}
{u'ErE-e': 3, u'rE-j': 9, u'rE-': 14}
{u'warE-je': 3, u'rE-': 2}
{u'ErE-e': 11, u'rE-j': 16, u'ErE-wa': 12, u'rE-': 12}
{u'ErE-e': 3, u'jErE-wa': 2}
{u'6rE-': 5, u'O.6rE-o.': 4}
{u'tirE-': 17, u'rE-': 18}
{u'virE-': 3, u'rE-': 1}
{u'6rE-': 34, u'6rE-E': 4}
{u'6rE-E': 2, u'6.6rE-9.': 1}
{u'irE-': 11, u'rE-': 13}
{u'j6rE-': 27, u'6rE-': 39}
{u'6rE-': 588, u'O.6rE-o.': 95}

(u'fi2S', u'pI2S')
{u'ra-': 13, u'ira-': 10}
{u'wara-je': 3, u'ra-': 2}
{u'ra-': 14, u'Era-e': 3, u'ra-j': 9}
{u'vira-': 3, u'ra-': 1}
{u'tira-': 17, u'ra-': 18}
{u'6ra-': 34, u'6ra-E': 4}
{u'Era-wa': 13, u'ra-': 12, u'Era-e': 11, u'ra-j': 16}
{u'ra-': 14, u'ra-j': 13}
{u'6ra-': 5, u'O.6ra-o.': 4}
{u'6ra-E': 2, u'6.6ra-9.': 1}
{u'6ra-': 588, u'O.6ra-o.': 95}
{u'6ra-': 39, u'j6ra-': 27}
{u'Era-e': 3, u'jEra-wa': 2}

(u'ps1P', u'fi1P')
{u'asj\xf4-6r\xf4': 7, u'j\xf4-6r\xf4': 1}
{u'j\xf4-ir\xf4': 7, u'j\xf4-6r\xf4': 69}
{u'j.

{u'E.6rj\xf4-6.\xe2': 81, u'6rj\xf4-\xe2': 31}
{u'rj\xf4-s\xe2': 6, u'irj\xf4-j\xe2': 3, u'irj\xf4-\xe2': 9}
{u'E.6rj\xf4-6.\xe2': 65, u'6rj\xf4-\xe2': 113}
{u'rj\xf4-s\xe2': 56, u'irj\xf4-j\xe2': 8}
{u'rj\xf4-\xe2': 5, u'rj\xf4-s\xe2': 5, u'rj\xf4-z\xe2': 22, u'rj\xf4-j\xe2': 2}
{u'tri\xf4-s\xe2': 13, u'ri\xf4-\xe2': 15}
{u'rj\xf4-j\xe2': 15, u'rj\xf4-z\xe2': 3}

(u'pi1S', u'pc2P')
{u'-6rje': 239, u'-drije': 9, u'-tirje': 5}
{u'j-rje': 32, u'-6rje': 40}
{u'Hi-6rje': 1, u'-vrije': 1}
{u'wa-6vrije': 9, u'-rje': 12}
{u'E-Orje': 2, u'-j6rje': 16, u'-trije': 27, u'-rje': 16}
{u'-6rje': 32, u'-tirje': 5}
{u'E-Orje': 1, u'E-airje': 1}
{u'-rje': 36, u'-vrije': 5}
{u'wa-Erje': 10, u'-rje': 9}
{u'-virje': 3, u'-6rje': 13}
{u'-6rje': 239, u'-drije': 1, u'-rje': 8}
{u'-drije': 40, u'-tirje': 7}
{u'-j6rje': 10, u'-rje': 11}
{u'-6rje': 6, u'-irje': 9}
{u'-drije': 2, u'-rje': 614}
{u'-6rje': 7, u'-mirje': 4}

(u'pc1P', u'fi2P')
{u'j\xf4-e': 51, u'rj\xf4-j6re': 39}

(u'pi2S', u'inf')
{u'-e': 180, u'E

{u'j\xf4-': 7, u'alj\xf4-o': 5}
{u'j\xf4-': 5, u'avj\xf4-E': 3}
{u'6.j\xf4-E.': 135, u'6.j\xf4-9.': 4}
{u'j\xf4-': 15, u'vj\xf4-': 21}
{u'j\xf4-': 51, u'O.j\xf4-o.': 6}
{u'j\xf4-': 650, u'\xf4-': 106}
{u'j\xf4-': 270, u'tj\xf4-': 43}
{u'j\xf4-': 70, u'sj\xf4-': 276}
{u'j\xf4-': 1, u'alj\xf4-vE': 2}
{u'EJ\xf4-\xea': 21, u'\xf4-': 6}
{u'j\xf4-': 4, u'Ej\xf4-je': 3, u'\xf4-': 3, u'Ej\xf4-wa': 3}
{u'6nj\xf4-j\xea': 27, u'6.j\xf4-E.': 1}
{u'j\xf4-': 43, u'dj\xf4-': 46}
{u'j\xf4-': 16, u'ij\xf4-': 221}
{u'j\xf4-': 478, u'O.j\xf4-o.': 84}
{u'j\xf4-': 5, u'O.j\xf4-o.': 9}
{u'j\xf4-': 21, u'\xf4-': 25, u'Ej\xf4-e': 14}

(u'ii1P', u'ai2P')
{u'j\xf4-yt': 9, u'j\xf4-at': 7}
{u'j\xf4-at': 41, u'zj\xf4-t': 6}
{u'j\xf4-at': 18, u'sj\xf4-t': 248}
{u'j\xf4-at': 5, u'avj\xf4-yt': 2}

(u'is1S', u'pi1S')
{u'alas-vE': 2, u'as-': 1}
{u'dis-': 46, u's-': 21}
{u'tis-': 19, u's-': 21}
{u'ys-wa': 18, u'ys-E': 10}
{u's-': 24, u'is-E': 19}
{u'O.as-o.': 63, u'as-': 546}
{u'jas-': 48, u'as-': 163}
{u's-': 5, u'is-'

{u'6.j\xf4-E.6rE': 54, u'j\xf4-6rE': 2}

(u'ai1P', u'inf')
{u'ym-Etr': 1, u'ym-uvwar': 1}
{u'im-war': 2, u'm-r': 5}
{u'im-r': 57, u'm-r': 81}
{u'ym-Etr': 7, u'ym-war': 5, u'ym-ir': 8}
{u'ym-Er': 2, u'ym-war': 2, u'ym-ir': 2, u'm-r': 5}
{u'im-Etr': 15, u'm-r': 4}
{u'ym-Etr': 1, u'ym-war': 2}
{u'm-r': 3, u'im-Er': 3}
{u'ym-avwar': 2, u'ym-6vwar': 7}
{u'ym-6vwar': 2, u'ym-Er': 1}
{u'im-war': 6, u'im-r': 3, u'm-r': 33}

(u'pP', u'ps1S')
{u'\xe2-': 48, u'l\xe2-j': 7}
{u'\xe2-': 592, u'O.\xe2-o.': 74}
{u'\xe2-': 2, u'u.\xe2-9.': 4}
{u'6.\xe2-E.': 1, u'6n\xe2-jEn': 27}
{u'\xe2-': 170, u'j\xe2-': 101}
{u'6.\xe2-E.': 7, u'6.\xe2-9.': 2}
{u'\xe2-': 7, u'j\xe2-': 5, u'Ej\xe2-wa': 3}

(u'ps1P', u'is1P')
zéro {u'Ej\xf4-ysj\xf4': 2}

(u'fi1S', u'is1P')
{u'6rE-asj\xf4': 27, u'6rE-isj\xf4': 6}

(u'ai2P', u'pi1S')
{u'it-E': 7, u't-': 3}

(u'pi2S', u'pI1P')
{u'-\xf4': 180, u'E.-6.\xf4': 151}
{u'i-j\xf4': 15, u'-\xf4': 10, u'-s\xf4': 27, u'-z\xf4': 2}
{u'-j\xf4': 13, u'wa-6v\xf4': 3}
{u'-j\xf4': 13, u'-z

{u'O.as-o.': 63, u'as-': 545}
{'-': 4, u'is-': 4, u'is-Et': 8}
{u'las-j': 2, u'as-': 1}
{'-': 52, u'is-': 28}
{'-': 1, u'is-wa': 1, u'is-': 18}
{u'ys-wav': 7, u'ys-wa': 1, u'ys-E': 1}
{u'ys-Ez': 1, u'ys-wav': 2}

(u'pi2P', u'is2P')
zéro {u'Et-fysje': 1}

(u'pi3S', u'ai1P')
{u'-m': 196, u'i-jam': 68}
{u'E-im': 21, u'-jam': 3, u'E-ym': 2}
{u'-tim': 5, u'-am': 76}
{u'-am': 7, u'-mim': 4}
{u'a-ym': 1, u'-tim': 9}
{u'-m': 28, u'i-jam': 4}
{u'-dim': 1, u'-am': 127}
{u'-am': 47, u'-im': 3}
{u'-dim': 3, u'-am': 158}
{u'-jam': 11, u'-vim': 12}
{u'-m': 25, u'i-ym': 2, u'i-jam': 15, u'-jam': 10}
{u'i-Ekym': 3, u'i-jam': 3}
{u'-am': 31, u'-im': 9}
{u'-m': 1, u'-jam': 1, u'-vim': 3}
{u'-am': 180, u'E.-6.am': 150}
{u'-jam': 7, u'E-ym': 2}
{u'-am': 31, u'-vim': 3}
{u'-dim': 6, u'-am': 38, u'-tim': 5}
{u'-jam': 20, u'wa-ym': 11}
{u'-jam': 13, u'E-ym': 12}
{u'-jam': 8, u'wa-ym': 8, u'wa-im': 8}
{u'-m': 5, u'-Ham': 17}
{u'-tim': 2, u'-wam': 19}
{u'-m': 14, u'-jam': 2}
{u'-dim': 36, u'-tim': 5}

(u'pc2P'


(u'pi3P', u'pI2P')
{u'-e': 213, u'E.-6.e': 163}
{u'9l-6je': 2, u'9.-u.e': 2}
{u'-je': 2, u'wa-Eje': 3}
{u'i-je': 211, u'-je': 21}
{u'-e': 99, u'z-t': 7}
{u'-e': 6, u'Ej-waje': 2}
{u'9.-6.e': 2, u'9.-u.e': 4}

(u'ii2P', u'is1S')
{u'je-as': 1713, u'je-is': 93}
{u'ije-as': 21, u'je-s': 9}
{u'e-is': 19, u'e-as': 16}
{u'je-as': 18, u'je-': 218}
{u'waje-is': 2, u'waje-ys': 1}
{u'je-as': 1, u'6nje-\xeas': 27}
{u'je-ys': 9, u'je-as': 10}
{u'ije-as': 201, u'e-as': 16}
{u'je-as': 3, u'je-ys': 2}
{u'je-as': 72, u'zje-s': 13}
{u'je-as': 32, u'E.je-y.': 12}

(u'pi1P', u'pi3P')
{u'O.\xf4-o.': 98, u'\xf4-': 588}
{u'j\xf4-': 4, u'\xf4-': 6, u'Ej\xf4-wa': 3}
{u'j\xf4-': 91, u'\xf4-': 164}
{u'6.\xf4-9.': 1, u'6.\xf4-E.': 39, u'6.\xf4-2.': 5}
{u'\xf4-': 1, u'al\xf4-v\xf4': 2}
{u'6.\xf4-9.': 4, u'6.\xf4-E.': 121}
{u'\xf4-': 25, u'u.\xf4-9.': 4}
{u'j\xf4-': 2, u'waj\xf4-Ej': 2}
{u'6.\xf4-E.': 1, u'6n\xf4-jEn': 10}

(u'ii1P', u'fi2S')
{u'6nj\xf4-j\xeadra': 27, u'6.j\xf4-E.6ra': 1}
{u'zj\xf4-ra': 39, u'j\xf

{u'ri\xf4-j\xf4': 15, u'tri\xf4-sj\xf4': 6}
{u'rj\xf4-j\xf4': 13, u'Er..-wa..': 1}
{u'r..-z..': 5, u'rj\xf4-j\xf4': 15}
{u'irj\xf4-j\xf4': 4, u'r..-s..': 4}
{u'E.6rj\xf4-6.j\xf4': 65, u'6rj\xf4-j\xf4': 114}
{u'rj\xf4-j\xf4': 5, u'Er..-wa..': 11}
{u'r..-s..': 1, u'rj\xf4-\xf4': 9}

(u'pi1P', u'pI2P')
{u'ul\xf4-6je': 2, u'\xf4-e': 2}
{u'av\xf4-Eje': 2, u'\xf4-e': 18}
{u'z\xf4-t': 7, u'\xf4-e': 100}
{u'waj\xf4-Eje': 2, u'\xf4-e': 2}
{u'Ej\xf4-waje': 2, u'\xf4-e': 7}

(u'pi1S', u'pi2P')
{u'-ze': 3, u'-we': 20}
{u'-e': 179, u'E.-6.e': 150}
{u'i-je': 5, u'-se': 1, u'-ve': 3}
{u'i-je': 20, u'-se': 37, u'-ve': 12, u'-je': 11}
{u'-e': 160, u'-de': 9, u'-ve': 3, u'-te': 5}
{u'-ze': 3, u'-je': 10}
{u'E-ave': 1, u'E-aise': 1}
{u'-e': 7, u'-me': 4}
{u'wa-6ve': 6, u'-je': 2, u'wa-Eje': 3}
{u'-e': 99, u'-te': 5}
{u'-ze': 4, u'-se': 56, u'-je': 10, u'i-je': 20}
{u'i-je': 11, u'-se': 39}
{u'-e': 135, u'-de': 1}
{u'wa-6ve': 3, u'-je': 13}
{u'-je': 1, u'-se': 5, u'-te': 15}
{u'-je': 39, u'-se': 3}
{u'-we

In [22]:
zeroPop={}
for paire in zeroEntropie:
    for t in rules[paire].classe:
        pop=0
        for c in rules[paire].classe[t]:
            pop+=rules[paire].classe[t][c]
        zeroPop[paire]=pop
    

In [34]:
stemspaceThresh=100
stemspaceRules=[]
for k in sorted(zeroPop, key=zeroPop.get,reverse=True):
    print k, zeroPop[k]
    if zeroPop[k]>=stemspaceThresh:
        stemspaceRules.append(k)

(u'pP', u'pI1P') 5283
(u'ppFP', u'ppMP') 5209
(u'ai3P', u'ai1S') 5071
(u'ai1S', u'ai3P') 5071
(u'ai2S', u'ai1P') 5007
(u'ai1P', u'ai2S') 5007
(u'ii1P', u'ii2P') 4865
(u'is1S', u'ai2S') 4555
(u'ai2S', u'is1S') 4555
(u'ai1P', u'ai1S') 4483
(u'ai1S', u'ai1P') 4483
(u'ai1P', u'is1S') 4391
(u'is1S', u'ai1P') 4391
(u'ai3P', u'is1S') 4048
(u'ps2P', u'ps1S') 3469
(u'ai2P', u'ai2S') 3420
(u'ai2S', u'ai2P') 3420
(u'ai1P', u'ai2P') 3386
(u'ai2P', u'ai1P') 3386
(u'is1S', u'ai2P') 3173
(u'ai2P', u'is1S') 3173
(u'ppFP', u'ai2P') 3147
(u'ai1S', u'ai2P') 3144
(u'ps1P', u'ps2P') 3098
(u'ps2P', u'ps1P') 3098
(u'is1P', u'ai2S') 2512
(u'ai2S', u'is1P') 2512
(u'ai1P', u'is1P') 2511
(u'is1P', u'ai1P') 2511
(u'is1S', u'is1P') 2500
(u'is1P', u'is1S') 2500
(u'ii1S', u'is1P') 2474
(u'pi1P', u'is1P') 2473
(u'pi2P', u'is1P') 2473
(u'ai2P', u'is1P') 2399
(u'is1P', u'ai2P') 2399
(u'is1P', u'ii1P') 2285
(u'ai3P', u'ai2S') 614
(u'ai2S', u'ai3P') 614
(u'ai1P', u'ai3P') 513
(u'ai3P', u'ai1P') 513
(u'is1S', u'ai3P') 502

In [35]:
len(stemspaceRules)

49

In [36]:
stemSpace=nx.Graph()
for c1,c2 in stemspaceRules:
    if (c2,c1) in stemspaceRules:
#        print c1,c2
        stemSpace.add_edge(c1,c2)
stemSpaceCliques=list(nx.find_cliques(stemSpace))
len(stemSpaceCliques),sorted(stemSpaceCliques,key=len,reverse=True)

(3,
 [[u'is1S', u'ai2P', u'ai1P', u'ai2S', u'ai1S', u'ai3P'],
  [u'is1S', u'ai2P', u'ai1P', u'ai2S', u'is1P'],
  [u'ps1P', u'ps2P']])

In [26]:
fSamplePMS="paradigmes.csv"
fSamplePMO="Morphomes-paradigmes.csv"

In [121]:
def openSample(numero,typeSample="PMS"):
    result=None
    fSample=""
    if typeSample=="PMS" and numero in samples:
        fSample=rep+samples[numero]+fSamplePMS
    elif typeSample=="PMO" and numero in samples:
        fSample=rep+samples[numero]+fSamplePMO
    if fSample:
        result=pd.read_csv(rep+samples[numero]+fSamplePMS,sep=";",index_col=0,keep_default_na=False,encoding="utf8")
    return result    

In [122]:
df=openSample(2,typeRules)

In [123]:
def makeZeros(sampleFormes,inFormes=True,contextFree=True):
    series=sampleFormes.copy()
    series={k:v for k,v in series.iteritems() if v!=""}

    paires=rules.keys()
    formesC={}
    for c in series:
        if not c in formesC:
            formesC[c]=set()
        formesC[c].add(series[c])
        if debug: print c, series[c]
        formeC=series[c]
        pairesC=[(c1,c2) for (c1,c2) in paires if c1==c]
        for p in pairesC:
            formeP=rules[p].sortirForme(series[c],contextFree=contextFree)
            if formeP and len(formeP)==1:
                p1=p[1]
                if debug: print p1,formeP
                if p1 not in formesC:
                    formesC[p1]=set()
                formesC[p1].add(formeP.keys()[0])
    if inFormes:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1}
    else:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1 and k not in series}

    return formesZero

In [124]:
def fillStemSpace(sampleFormes,inFormes=True,contextFree=True):
    series=sampleFormes.copy()
    series={k:v for k,v in series.iteritems() if v!=""}

    paires=stemspaceRules
    formesC={}
    for c in series:
        if not c in formesC:
            formesC[c]=set()
        formesC[c].add(series[c])
        if debug: print c, series[c]
        formeC=series[c]
        pairesC=[(c1,c2) for (c1,c2) in paires if c1==c]
        for p in pairesC:
            formeP=rules[p].sortirForme(series[c],contextFree=contextFree)
            if formeP and len(formeP)==1:
                p1=p[1]
                if debug: print p1,formeP
                if p1 not in formesC:
                    formesC[p1]=set()
                formesC[p1].add(formeP.keys()[0])
    if inFormes:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1}
    else:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1 and k not in series}

    return formesZero

In [125]:
cases=[c for c in df.columns.tolist() if c!="lexeme"]
cases=sorted(cases)

In [126]:
lexemes=u"clore éclore falloir valoir".split(" ")
lexemes=[l for l in df["lexeme"].tolist() if not l.endswith(u"er")]
lexemes

[u'abasourdir',
 u'abattre',
 u'abolir',
 u'aboutir',
 u'abrutir',
 u'absoudre',
 u'abstenir',
 u'abstraire',
 u'ab\xe2tardir',
 u'ab\xeatir',
 u'accomplir',
 u'accourcir',
 u'accourir',
 u'accroire',
 u'accroupir',
 u'accro\xeetre',
 u'accueillir',
 u'acqu\xe9rir',
 u'adjoindre',
 u'admettre',
 u'adoucir',
 u'advenir',
 u'affadir',
 u'affaiblir',
 u'affermir',
 u'affranchir',
 u'agir',
 u'agonir',
 u'agrandir',
 u'aguerrir',
 u'ahurir',
 u'aigrir',
 u'alanguir',
 u'alentir',
 u'alourdir',
 u'alunir',
 u'amaigrir',
 u'amerrir',
 u'amincir',
 u'amoindrir',
 u'amollir',
 u'amortir',
 u'anoblir',
 u'an\xe9antir',
 u'apercevoir',
 u'aplanir',
 u'aplatir',
 u'appara\xeetre',
 u'appartenir',
 u'appauvrir',
 u'appendre',
 u'appesantir',
 u'applaudir',
 u'apprendre',
 u'approfondir',
 u'arrondir',
 u'assagir',
 u'assaillir',
 u'assavoir',
 u'asseoir',
 u'asservir',
 u'assombrir',
 u'assortir',
 u'assoupir',
 u'assouplir',
 u'assourdir',
 u'assouvir',
 u'assujettir',
 u'astreindre',
 u'atteindr

In [127]:
dfNew=df.copy()
for lexeme in lexemes:
    index=df.loc[df['lexeme']==lexeme].index.values
    if index:
        i=index[0]
        sampleFormes=df.iloc[i].to_dict()
        sampleFormes={k:v for k,v in sampleFormes.iteritems() if v!=""}
        lexemeI=sampleFormes["lexeme"]
        print lexemeI,
#        formesZero=makeZeros(sampleFormes,inFormes=False)
        formesZero=fillStemSpace(sampleFormes,inFormes=False,contextFree=not avecContexte)
        for case in cases:
            if case in sampleFormes:
#                print "\t",case,"\t  ",sampleFormes[case]
                continue
            elif case in formesZero:
#                print "\t",case,"\t=>",formesZero[case]
                dfNew.iloc[i][case]=formesZero[case]
            else:
#                print "\t",case
                continue
#        print

abasourdir abattre abolir aboutir abrutir absoudre abstenir abstraire abâtardir abêtir accomplir accourcir accourir accroire accroupir accroître accueillir acquérir adjoindre admettre adoucir advenir affadir affaiblir affermir affranchir agir agonir agrandir aguerrir ahurir aigrir alanguir alentir alourdir alunir amaigrir amerrir amincir amoindrir amollir amortir anoblir anéantir apercevoir aplanir aplatir apparaître appartenir appauvrir appendre appesantir applaudir apprendre approfondir arrondir assagir assaillir assavoir asseoir asservir assombrir assortir assoupir assouplir assourdir assouvir assujettir astreindre atteindre attendre attendrir atterrir attiédir avachir avertir avilir avoir bannir barrir battre blanchir bleuir blondir blottir blêmir boire bondir bouffir bouillir brandir bruire brunir bâtir bénir calmir candir ceindre chancir choir choisir chérir circoncire circonscrire circonvenir clapir clore combattre commettre comparaître compatir complaire comprendre compromettre

## Ouvrir les fichiers de morphomes

In [200]:
morphomeCases=pd.read_pickle(rep+samples[54]+"Morphomes.pkl")[["case","morphome"]].drop_duplicates().to_dict()
casesMC=morphomeCases["case"]
morphomesMC=morphomeCases["morphome"]
listMorphomes=[]
for element in casesMC:
    listMorphomes.append(morphomesMC[element].split("/"))
cases

[u'ai1P',
 u'ai1S',
 u'ai2P',
 u'ai2S',
 u'ai3P',
 u'ai3S',
 u'fi1P',
 u'fi1S',
 u'fi2P',
 u'fi2S',
 u'fi3P',
 u'fi3S',
 u'ii1P',
 u'ii1S',
 u'ii2P',
 u'ii2S',
 u'ii3P',
 u'ii3S',
 u'inf',
 u'is1P',
 u'is1S',
 u'is2P',
 u'is2S',
 u'is3P',
 u'is3S',
 u'pI1P',
 u'pI2P',
 u'pI2S',
 u'pP',
 u'pc1P',
 u'pc1S',
 u'pc2P',
 u'pc2S',
 u'pc3P',
 u'pc3S',
 u'pi1P',
 u'pi1S',
 u'pi2P',
 u'pi2S',
 u'pi3P',
 u'pi3S',
 u'ppFP',
 u'ppFS',
 u'ppMP',
 u'ppMS',
 u'ps1P',
 u'ps1S',
 u'ps2P',
 u'ps2S',
 u'ps3P',
 u'ps3S']

In [196]:
def makeTabularParadigme(lexeme,lDF,dictColours,title="",coulLim=False, cat="V",dictMorphomes={}):
    row=lDF[lDF["lexeme"]==lexeme]
    tabular=[]
    def makeValue(case):
        print case,row[case],len(row[case]),row[case].values
        if len(row[case])>0 and len(row[case].values[0])>0:
            result=coderIPA(row[case].values[0])
        elif dictMorphomes!={} and case in dictMorphomes:
            altCase=dictMorphomes[case][0]
            if len(row[altCase])>0 and len(row[altCase].values[0])>0:
                result=coderIPA(row[altCase].values[0])
            else:
                result="---"
        else:
            result="---"
    def makeLine6(tenseCode):
        line=[tabTemps[tenseCode]]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            case=tenseCode+person
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
        return r" & ".join(line)+r"\\"

    def makeLine3(tenseCode):
        line=[tabTemps[tenseCode]]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            if person in ["2S","1P","2P"]:
                case=tenseCode+person
                if case in dictColours:
                    line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
                else:
                    line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
#                line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
            else:
                line.append(r"---")
        return r" & ".join(line)+r"\\"
    
    def makeLineNF():
        line=["non-fini"]
        for case in ["inf","pP","ppMS","ppMP","ppFS","ppFP"]:
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
#            line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
        return r" & ".join(line)+r"\\"

    def makeLineMF(nombre):
        line=[]
        for genre in "mf":
            case=genre+nombre
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
        return r" & ".join(line)+r"\\"
    
    def makeLineCoulLim():
        line=[]
        for numLimite,limite in enumerate(listLimites):
            line.append(r"\cellcolor{%s}%s"%(listLimCoul[numLimite],"$<$"+str(limite)))
        return r"\hline\hline "+r" & ".join(line)+r"\\"
        
    if cat=="V":
        top=[
            r"\begin{center}",
            r"\begin{tabular}{ccccccc}",
            r"\toprule",
            " & ".join([lexeme]+[p+n for n in ["sg","pl"] for p in "123" ])+r"\\",
            r"\midrule"
            ]
        bottom=[
            r"\bottomrule",
            r"\end{tabular}\\",
            title,
            r"\end{center}",
            r"\bigskip",
            r""
            ]
        tabular.append("\n".join(top))
        for tenseCode in ["pi","ii","fi","pc", "ps","ai", "is"]:
            tabular.append(makeLine6(tenseCode))
        tabular.append(makeLine3("pI"))
        tabular.append(makeLineNF())
    elif cat=="A":
        top=[
            r"\begin{center}",
            r"\begin{tabular}{cc}",
            r"\toprule"
            ]
        bottom=[
            r"\bottomrule",
            r"\end{tabular}\\",
            title,
            r"\end{center}",
            r"\bigskip",
            r""
            ]
        tabular.append("\n".join(top))
        for number in "sp":
            tabular.append(makeLineMF(number))
    if coulLim:
        tabular.append(makeLineCoulLim())
    tabular.append("\n".join(bottom))
    return "\n".join(tabular)

In [162]:
etColours=["orange",
           "brown!50",
           "brown",
           "blue!10",
           "yellow!50",
           "blue!25",
           "teal!50",
           "blue!50",
           "cyan!50",
           "lime",
           "pink",
           "magenta!50",
          ]
etCells=[
    ["pi1P","pi2P"]+["ii"+p+n for p in "123" for n in "SP"],
    ["pi3P"],
    ["pi"+p+"S" for p in "123"],
    ["pP"],
    ["pI2S"],
    ["pI1P","pI2P"],
    ["ps"+p+"S" for p in "123"]+["ps3P"],
    ["ps1P","ps2P"],
    ["inf"],
    ["fi"+p+n for p in "123" for n in "SP"]+["pc"+p+n for p in "123" for n in "SP"],
    ["ai"+p+n for p in "123" for n in "SP"]+["is"+p+n for p in "123" for n in "SP"],
    ["pp"+g+n for g in "MF" for n in "SP"],
        ]
tabTemps={
    "pi":u"ind. prs",
    "ii":u"ind. ipf",
    "ai":u"ind. ps",
    "fi":u"ind. fut",
    "ps":u"subj. prs",
    "is":u"subj. ipf",
    "pc":u"cond. prs",
    "pI":u"imper. prs",
    "inf":u"non-fini"
    }
dictEtColours={}
for nET,ET in enumerate(etCells):
    for c in ET:
        dictEtColours[c]=etColours[nET]
dictEtColours

{'ai1P': 'pink',
 'ai1S': 'pink',
 'ai2P': 'pink',
 'ai2S': 'pink',
 'ai3P': 'pink',
 'ai3S': 'pink',
 'fi1P': 'lime',
 'fi1S': 'lime',
 'fi2P': 'lime',
 'fi2S': 'lime',
 'fi3P': 'lime',
 'fi3S': 'lime',
 'ii1P': 'orange',
 'ii1S': 'orange',
 'ii2P': 'orange',
 'ii2S': 'orange',
 'ii3P': 'orange',
 'ii3S': 'orange',
 'inf': 'cyan!50',
 'is1P': 'pink',
 'is1S': 'pink',
 'is2P': 'pink',
 'is2S': 'pink',
 'is3P': 'pink',
 'is3S': 'pink',
 'pI1P': 'blue!25',
 'pI2P': 'blue!25',
 'pI2S': 'yellow!50',
 'pP': 'blue!10',
 'pc1P': 'lime',
 'pc1S': 'lime',
 'pc2P': 'lime',
 'pc2S': 'lime',
 'pc3P': 'lime',
 'pc3S': 'lime',
 'pi1P': 'orange',
 'pi1S': 'brown',
 'pi2P': 'orange',
 'pi2S': 'brown',
 'pi3P': 'brown!50',
 'pi3S': 'brown',
 'ppFP': 'magenta!50',
 'ppFS': 'magenta!50',
 'ppMP': 'magenta!50',
 'ppMS': 'magenta!50',
 'ps1P': 'blue!50',
 'ps1S': 'teal!50',
 'ps2P': 'blue!50',
 'ps2S': 'teal!50',
 'ps3P': 'teal!50',
 'ps3S': 'teal!50'}

In [197]:
print makeTabularParadigme(u"clore",dfNew,dictEtColours,dictMorphomes=dictCaseMorphome)

pi1S 775    klo
Name: pi1S, dtype: object 1 [u'klo']
pi2S 775    
Name: pi2S, dtype: object 1 [u'']
pi3S 775    klo
Name: pi3S, dtype: object 1 [u'klo']
pi1P 775    
Name: pi1P, dtype: object 1 [u'']
pi2P 775    
Name: pi2P, dtype: object 1 [u'']
pi3P 775    
Name: pi3P, dtype: object 1 [u'']
ii1S 775    
Name: ii1S, dtype: object 1 [u'']
ii2S 775    
Name: ii2S, dtype: object 1 [u'']
ii3S 775    
Name: ii3S, dtype: object 1 [u'']
ii1P 775    
Name: ii1P, dtype: object 1 [u'']
ii2P 775    
Name: ii2P, dtype: object 1 [u'']
ii3P 775    
Name: ii3P, dtype: object 1 [u'']
fi1S 775    
Name: fi1S, dtype: object 1 [u'']
fi2S 775    
Name: fi2S, dtype: object 1 [u'']
fi3S 775    
Name: fi3S, dtype: object 1 [u'']
fi1P 775    
Name: fi1P, dtype: object 1 [u'']
fi2P 775    
Name: fi2P, dtype: object 1 [u'']
fi3P 775    
Name: fi3P, dtype: object 1 [u'']
pc1S 775    
Name: pc1S, dtype: object 1 [u'']
pc2S 775    
Name: pc2S, dtype: object 1 [u'']
pc3S 775    
Name: pc3S, dtype: object 1 [u'']
p

In [134]:
lexeme=u"extraire"
ix=df.loc[df.lexeme==lexeme,:].T.columns.values[0]
pd.concat([df.loc[df.lexeme==lexeme,:].T.rename(columns={ix:'Old'}),
dfNew.loc[dfNew.lexeme==lexeme,:].T.rename(columns={ix:'New'})],
         axis=1)

,Old,New
lexeme,extraire,extraire
ai1P,,
ai1S,,
ai2P,,
ai2S,,
ai3P,,
ai3S,,
fi1P,,
fi1S,,
fi2P,,


In [59]:
dfNew.loc[dfNew.lexeme==u"valoir",:].T

,4262
lexeme,valoir
ai1P,valym
ai1S,valy
ai2P,
ai2S,valy
ai3P,valyr
ai3S,valy
fi1P,
fi1S,
fi2P,


In [42]:
for i in range(500):
    sampleFormes=df.loc[i].to_dict()
    lexemeI=sampleFormes["lexeme"]
    if not lexemeI.endswith("er"):
        print lexemeI
        numero=1
        for k,v in sampleFormes.iteritems():
            if k!="lexeme" and v!="":
                print numero,k,v
                numero+=1
        print
        formesZero=makeZeros(sampleFormes,inFormes=False)
        for k,v in formesZero.iteritems():
            if k!="lexeme":
                print numero,k,v
                numero+=1
        print

abasourdir
1 pP abazurdisâ
2 ai3S abazurdi
3 ai3P abazurdir
4 ps3S abazurdis
5 ii3S abazurdisE
6 inf abazurdir
7 ii3P abazurdisE
8 pi2S abazurdi
9 pi2P abazurdise
10 ppFS abazurdi
11 pI2S abazurdi
12 fi2P abazurdire
13 pi1S abazurdi
14 pi3P abazurdis
15 pi3S abazurdi
16 ppMS abazurdi
17 pc3S abazurdirE
18 pc3P abazurdirE
19 fi3S abazurdira
20 fi3P abazurdirô
21 ppMP abazurdi
22 fi1S abazurdirE
23 pc1S abazurdirE

24 pI1P abazurdisô
25 ii1S abazurdisE
26 ii2S abazurdisE
27 pi1P abazurdisô
28 ps2P abazurdje
29 ppFP abazurdi
30 is2S abazurdis
31 pc1P abazurdirjô
32 is1S abazurdis
33 ai2S abazurdi
34 ps2S abazurdis
35 is3P abazurdis
36 is3S abazurdi
37 ai1P abazurdim
38 ai1S abazurdi
39 pI2P abazurdise
40 ps3P abazurdis
41 pc2P abazurdirje
42 ps1S abazurdis

abattre
1 ii1P abatjô
2 ii1S abatE
3 pP abatâ
4 ai3S abati
5 ai3P abatir
6 ps3S abat
7 ii3S abatE
8 inf abatr
9 ii3P abatE
10 pi2S aba
11 pi2P abate
12 ppFS abaty
13 ppFP abaty
14 fi2P abatre
15 pI2P abate
16 ps2S abat
17 ii2P abatje
1

2 ii1S admEtE
3 pP admEtâ
4 ai3S admi
5 ps3S admEt
6 ii3S admEtE
7 inf admEtr
8 ii3P admEtE
9 pi2S admE
10 ai1S admi
11 pi2P admEte
12 ppFS admiz
13 ppFP admiz
14 pI2S admE
15 fi2P admEtre
16 fi2S admEtra
17 pI2P admEte
18 ps2S admEt
19 ii2P admEtje
20 pi1S admE
21 pi1P admEtô
22 pi3P admEt
23 pi3S admE
24 ppMS admi
25 pc3S admEtrE
26 pc3P admEtrE
27 fi3S admEtra
28 fi3P admEtrô
29 ppMP admi
30 fi1P admEtrô
31 fi1S admEtrE
32 pc1S admEtrE
33 pI1P admEtô

34 ps2P admEsje
35 ai3P admir
36 ai2S admi
37 pc2P admâdrije
38 is3S admi
39 ai1P admim
40 ps3P admEt
41 ii2S admEtE
42 ps1S admEt

adoucir
1 ii1S adusisE
2 pP adusisâ
3 ai3S adusi
4 ai3P adusir
5 ps3S adusis
6 ii3S adusisE
7 inf adusir
8 ii3P adusisE
9 pi2S adusi
10 pi2P adusise
11 ppFS adusi
12 ppFP adusi
13 pI2S adusi
14 fi2P adusire
15 fi2S adusira
16 pi1S adusi
17 pi3P adusis
18 pi3S adusi
19 ppMS adusi
20 pc3S adusirE
21 pc3P adusirE
22 fi3S adusira
23 fi3P adusirô
24 ppMP adusi
25 fi1P adusirô
26 fi1S adusirE
27 pc1S adusirE

28

42 ps1S amEgris

amerrir
1 pP amErjâ
2 ai3S amErja
3 ai3P amErjEr
4 ii3S amErjE
5 inf amErje
6 ii3P amErjE
7 pi2S amEri
8 ai1S amErjE
9 pi2P amErje
10 ppFS amErje
11 ppFP amErje
12 pI2S amEri
13 fi2P amErire
14 fi2S amErira
15 pi1S amEri
16 pi1P amErjô
17 pi3P amEri
18 pi3S amEri
19 ppMS amErje
20 pc3S amErirE
21 fi3S amErira
22 ppMP amErje
23 fi1P amErirô
24 fi1S amErirE
25 pc1S amErirE

26 ii1S amErjE
27 ii2S amErjE
28 ps2S amEri
29 is2S amErjas
30 is1S amErjas
31 pc1P amErrjô
32 ai2S amErja
33 is3P amErjas
34 is3S amErja
35 ai1P amErjam
36 ps3S amEri
37 ps3P amEri
38 pI2P amErje
39 pI1P amErjô
40 ps1S amEri

amincir
1 ii1S amêsisE
2 pP amêsisâ
3 ai3S amêsi
4 ai3P amêsir
5 ps3S amêsis
6 ii3S amêsisE
7 inf amêsir
8 ii3P amêsisE
9 pi2S amêsi
10 pi2P amêsise
11 ppFS amêsi
12 ppFP amêsi
13 pI2S amêsi
14 fi2P amêsire
15 pi1S amêsi
16 pi3P amêsis
17 pi3S amêsi
18 ppMS amêsi
19 pc3S amêsirE
20 pc3P amêsirE
21 fi3S amêsira
22 fi3P amêsirô
23 ppMP amêsi
24 fi1S amêsirE
25 pc1S amêsirE

26 pI1

26 pi1P aprOfôdisô
27 ps2P aprOfôdje
28 ps2S aprOfôdis
29 is2S aprOfôdis
30 pc1P aprOfôdirjô
31 ai3S aprOfôdi
32 is1S aprOfôdis
33 ai2S aprOfôdi
34 is3P aprOfôdis
35 is3S aprOfôdi
36 ai1P aprOfôdim
37 ai1S aprOfôdi
38 pI2P aprOfôdise
39 ps3P aprOfôdis
40 pc2P aprOfôdirje
41 ps1S aprOfôdis

arrondir
1 pP arôdisâ
2 ai3S arôdi
3 ai3P arôdir
4 ps3S arôdis
5 ii3S arôdisE
6 inf arôdir
7 ii3P arôdisE
8 pi2S arôdi
9 ppFS arôdi
10 ppFP arôdi
11 fi2P arôdire
12 fi2S arôdira
13 pi1S arôdi
14 pi3P arôdis
15 pi3S arôdi
16 ppMS arôdi
17 pc3S arôdirE
18 fi3S arôdira
19 fi3P arôdirô
20 ppMP arôdi
21 fi1P arôdirô
22 fi1S arôdirE

23 pI1P arôdisô
24 ii1S arôdisE
25 ii2S arôdisE
26 pi1P arôdisô
27 ps2P arôdje
28 ps2S arôdis
29 is2S arôdis
30 pc1P arôdirjô
31 is1S arôdis
32 ai2S arôdi
33 is3P arôdis
34 is3S arôdi
35 ai1P arôdim
36 ai1S arôdi
37 pI2P arôdise
38 pI2S arôdi
39 ps3P arôdis
40 pc2P arôdirje
41 ps1S arôdis

assagir
1 ii1P asaZisjô
2 ii1S asaZisE
3 pP asaZisâ
4 ai3S asaZi
5 ai3P asaZir
6 is3S as

13 pi3S atâdri
14 ppMS atâdri
15 pc3S atâdrirE
16 pc3P atâdrirE
17 ppMP atâdri

18 pI1P atâdrisô
19 ii2S atâdrisE
20 pi1P atâdrisô
21 ps2P atâdrje
22 ppFP atâdri
23 is2S atâdris
24 is1S atâdris
25 pc1P atâdrirjô
26 ai2S atâdri
27 ps2S atâdris
28 is3P atâdris
29 is3S atâdri
30 ps3S atâdris
31 ai1P atâdrim
32 ps3P atâdris
33 pI2P atâdrise
34 pI2S atâdri
35 ai1S atâdri
36 pc2P atâdrirje
37 ps1S atâdris

atterrir
1 pP atErisâ
2 ai3S atEri
3 ai3P atErir
4 ps3S atEris
5 ai1P atErim
6 inf atErir
7 ii3P atErisE
8 pi2S atEri
9 fi2P atErire
10 fi2S atErira
11 pi1S atEri
12 pi3P atEris
13 pi3S atEri
14 ppMS atEri
15 pc3S atErirE
16 fi3S atErira
17 fi3P atErirô
18 ppMP atEri
19 fi1P atErirô
20 fi1S atErirE
21 pc1S atErirE

22 pI1P atErisô
23 ii1S atErisE
24 ii2S atErisE
25 pi1P atErisô
26 ps2S atEris
27 is2S atEris
28 is1S atEris
29 ai2S atEri
30 is3P atEris
31 is3S atEri
32 ii3S atErisE
33 ai1S atEri
34 pI2P atErise
35 pI2S atEri
36 ps3P atEris
37 ps1S atEris

attiédir
1 ii1P atjEdisjô
2 ii1S atj

In [ ]:
rules[("pi3S","ii1S")].sortirForme("Xi",contextFree=True)